In [ ]:
!pip install agno mem0ai opik

## Memory Utilization without Mem0AI API Key using Gemini

In [ ]:
!pip install langchain langchain-community fastembed

In [ ]:
import os
from google.colab import userdata

In [ ]:
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from mem0 import Memory
from langchain_community.embeddings import FastEmbedEmbeddings

In [ ]:
embeddings = FastEmbedEmbeddings(model_name = "jinaai/jina-embeddings-v2-base-en", max_length = 768)

In [ ]:
config = {
    "llm": {
        "provider": "gemini",
        "config": {
            "model": "gemini-2.5-flash-lite",
            "temperature": 0.8,
        }
    },
    "vector_store": {
        "provider": "qdrant",
        "config": {
            "collection_name": "devfestbaroda",
            "path": "/tmp/db",
            "embedding_model_dims": 768,
        }
    },
    "embedder": {
        "provider": "langchain",
        "config": {
            "model": embeddings
        }
    }
}

In [ ]:
client = Memory.from_config(config)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
messages = [
    {"role": "user", "content": "What is the must try food in Baroda"},
    {"role": "assistant", "content": "Sev Usal is must"},
    {"role": "user", "content": "I'm not into street food, I prefer Gujarati thalis."},
    {"role": "assistant", "content": "Head to Mandap in Baroda, it’s famous for authentic Gujarati thalis."},
]

In [ ]:
result1 = client.add(messages, user_id="personal", metadata={"category": "food"})

In [ ]:
result1

{'results': [{'id': '501ff9ef-c9ff-4c6b-9454-218ad42a9957',
   'memory': 'Prefers Gujarati thalis over street food',
   'event': 'ADD'},
  {'id': '4afb95a9-fc53-48d0-ac87-de3f30ab5744',
   'memory': 'Mandap in Baroda is famous for authentic Gujarati thalis',
   'event': 'ADD'}]}

In [ ]:
messages2 = [
    {"role": "user", "content": "I'm planning to travel to Hong Kong which Airlines to use from Bangalore"},
    {"role": "assistant", "content": "Cathay Pacific is the best option and have the direct flights. Any preferences?"},
    {"role": "user", "content": "Yes, I need Hindu Vegetarian meal and prefer window seat or person seat"},
    {"role": "assistant", "content": "Sure, I will got it. Do you like do add anything else?"},
]

In [ ]:
result2 = client.add(messages2, user_id="personal", metadata={"category": "travel"})

In [ ]:
result2

{'results': [{'id': '5ce327bd-48ad-4e08-bd79-3de11f0dee21',
   'memory': 'Planning to travel to Hong Kong from Bangalore',
   'event': 'ADD'},
  {'id': 'f2b28d67-e11c-44b7-928b-60fedc579b41',
   'memory': 'Needs Hindu Vegetarian meal',
   'event': 'ADD'},
  {'id': 'e3272238-6610-461d-a871-724ff81b5167',
   'memory': 'Prefers window seat or aisle seat',
   'event': 'ADD'}]}

In [ ]:
query = "I am travelling to Baroda, suggest me the dish to try and recommend place to visit"
query2 = "travelling to China, need help"

In [ ]:
memories = client.search(query2,user_id="personal",limit=30)

In [ ]:
memories['results']

[{'id': '5ce327bd-48ad-4e08-bd79-3de11f0dee21',
  'memory': 'Planning to travel to Hong Kong from Bangalore',
  'hash': 'dfbc94215c84af70a8caf9f541a59397',
  'metadata': {'category': 'travel'},
  'score': 0.7801442807204011,
  'created_at': '2025-10-03T23:30:41.797839-07:00',
  'updated_at': None,
  'user_id': 'personal'},
 {'id': 'e3272238-6610-461d-a871-724ff81b5167',
  'memory': 'Prefers window seat or aisle seat',
  'hash': 'f4f82f42fba68952a7d252041962ebc3',
  'metadata': {'category': 'travel'},
  'score': 0.6743861260705318,
  'created_at': '2025-10-03T23:30:41.849402-07:00',
  'updated_at': None,
  'user_id': 'personal'},
 {'id': 'f2b28d67-e11c-44b7-928b-60fedc579b41',
  'memory': 'Needs Hindu Vegetarian meal',
  'hash': '947effb400339f56d72a0d4ab3ebeb24',
  'metadata': {'category': 'travel'},
  'score': 0.6687318934645909,
  'created_at': '2025-10-03T23:30:41.826647-07:00',
  'updated_at': None,
  'user_id': 'personal'},
 {'id': '501ff9ef-c9ff-4c6b-9454-218ad42a9957',
  'memory

In [ ]:
context = "\n".join(f"- {m['memory']}" for m in memories['results'])

In [ ]:
context

'- Planning to travel to Hong Kong from Bangalore\n- Prefers window seat or aisle seat\n- Needs Hindu Vegetarian meal\n- Prefers Gujarati thalis over street food\n- Mandap in Baroda is famous for authentic Gujarati thalis'

## Trace and Monitor

In [ ]:
from opik import track, opik_context
from google.genai import Client

In [ ]:
os.environ['OPIK_API_KEY'] = userdata.get("OPIK_API_KEY")
os.environ['OPIK_WORKSPACE'] = "tarun-r-jain"
os.environ['OPIK_PROJECT_NAME'] = "memory"

In [ ]:
llm_client = Client()

In [ ]:
SYSTEM_PROMPT = """
You are an expert executive assistant who thinks carefully before responding,
adapting to the poliet communication style based on the previous user's established PREFERENCES and the complexity of their query.

Maintain a polished, professional tone that is warm yet efficient—concise for
simple questions, moderate for complex topics, and comprehensive for open-ended discussions.

Act as a trusted advisor who doesn't just answer questions but adds value through insights, anticipates needs,
and prioritizes what matters most while respecting the user's time with clear, actionable responses.
"""

In [ ]:
@track
def get_llm_response(query: str, user_id: str) -> str:
    memories = client.search(query,user_id=user_id,limit=30)
    mem_results = memories['results']
    context = "\n".join(f"- {m['memory']}" for m in mem_results)

    USER_PROMPT = f"""
      <question>
      QUESTION: {query}
      </question>

      <PREFERENCE>
      Preference: {context}
      </PREFERENCE>
    """

    response = llm_client.models.generate_content(
        model="gemini-2.5-flash",
        contents=USER_PROMPT,
        config={
            "system_instruction": SYSTEM_PROMPT
            }
    )

    opik_context.update_current_trace(
        metadata={
        "memories": [
            {
                "id": result["id"],
                "memory": result["memory"],
                "category": result["metadata"]["category"],
                "score": result["score"]
            }
            for result in mem_results
        ]
      }
    )

    return response.text

In [ ]:
user_query = "i need food and place recommendation for the food in New York"

In [ ]:
get_llm_response(user_query, user_id="personal")

OPIK: Started logging traces to the "memory" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0199adea-97ef-7281-b95c-1d085779d033&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


'Certainly! Given your preference for authentic Hindu Vegetarian Gujarati thalis, akin to the experience at Mandap in Baroda, and your desire for a window or aisle seat, finding the perfect spot in New York will require a focused approach. While New York boasts a diverse Indian culinary scene, pure Gujarati thali establishments, matching the exact "home-style" authenticity of places like Mandap, can be a bit more niche.\n\nHowever, I can recommend some excellent options that come very close or offer high-quality, pure vegetarian Indian dining that may appeal to your palate:\n\n### Food & Place Recommendations in New York:\n\n1.  **Satya (formerly Satya Foods / Vadilal Dining Hall)**\n    *   **Cuisine:** Primarily Gujarati and North Indian vegetarian.\n    *   **Why it fits:** This is one of the strongest contenders for authentic Gujarati thalis in New York. They offer a diverse, unlimited thali experience with various subzis, dal, kadhi, roti, rice, and often a sweet, mirroring the tr

## Memory Utilization with Mem0AI API key

In [ ]:
import os
from mem0 import MemoryClient

In [ ]:
os.environ['MEM0_API_KEY'] = userdata.get('MEM0_API_KEY')

In [ ]:
mem_client = MemoryClient()

In [ ]:
new_mess1 = [
    {"role": "user", "content": "What is the must try food in Baroda"},
    {"role": "assistant", "content": "Sev Usal is must"},
    {"role": "user", "content": "I'm not into street food, I prefer Gujarati thalis."},
    {"role": "assistant", "content": "Head to Mandap in Baroda, it’s famous for authentic Gujarati thalis."},
]
mem_client.add(new_mess1, user_id="tarun")

/usr/local/lib/python3.12/dist-packages/mem0/client/utils.py:38: DeprecationWarning: output_format='v1.0' is deprecated therefore setting it to 'v1.1' by default. Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  return func(*args, **kwargs)


{'results': []}

In [ ]:
new_mess2 = [
    {"role": "user", "content": "I'm planning to travel to Hong Kong which Airlines to use from Bangalore"},
    {"role": "assistant", "content": "Cathay Pacific is the best option and have the direct flights. Any preferences?"},
    {"role": "user", "content": "Yes, I need Hindu Vegetarian meal and prefer window seat or person seat"},
    {"role": "assistant", "content": "Sure, I will got it. Do you like do add anything else?"},
]
mem_client.add(new_mess2, user_id="tarun")

{'results': []}

In [ ]:
query = "I'm travelling to Europe- Vienna. Any food place recommendations?"
res = mem_client.search(query,user_id = "tarun")

In [ ]:
res

[{'id': '6bb35c4d-287a-44e0-b02f-1e39794d4867',
  'memory': 'User Needs Hindu Vegetarian meal',
  'user_id': 'tarun',
  'metadata': None,
  'categories': ['user_preferences', 'food'],
  'created_at': '2025-10-03T13:36:56.076879-07:00',
  'updated_at': '2025-10-03T13:36:56.103375-07:00',
  'expiration_date': None,
  'structured_attributes': {'day': 3,
   'hour': 20,
   'year': 2025,
   'month': 10,
   'minute': 36,
   'quarter': 4,
   'is_weekend': False,
   'day_of_week': 'friday',
   'day_of_year': 276,
   'week_of_year': 40},
  'score': 0.4946957354749122},
 {'id': '60598a8d-b34a-4a81-a2bf-a704c2f3a899',
  'memory': 'User Does not like street food',
  'user_id': 'tarun',
  'metadata': None,
  'categories': ['user_preferences', 'food'],
  'created_at': '2025-10-03T13:35:48.901778-07:00',
  'updated_at': '2025-10-03T13:35:48.971657-07:00',
  'expiration_date': None,
  'structured_attributes': {'day': 3,
   'hour': 20,
   'year': 2025,
   'month': 10,
   'minute': 35,
   'quarter': 4,
 

In [ ]:
new_context = "\n".join(f"- {m['memory']}" for m in res)

In [ ]:
new_context

'- User Needs Hindu Vegetarian meal\n- User Does not like street food\n- User Is planning to travel to Hong Kong from Bangalore\n- User Prefers Gujarati thalis over street food\n- User Prefers window seat or person seat'

In [ ]:
USER_PROMPT2 = f"""
<question>
QUESTION: {query}
</question>

<PREFERENCE>
Preference: {new_context}
</PREFERENCE>
"""

In [ ]:
llm_client = Client()

In [ ]:
llm_response = llm_client.models.generate_content(
    model="gemini-2.5-flash",
    contents=USER_PROMPT2,
    config={
        "system_instruction": SYSTEM_PROMPT
    }
)

In [ ]:
print(llm_response.text)

Certainly! Vienna offers a delightful culinary scene, and with your preference for Hindu Vegetarian meals in a sit-down restaurant setting, avoiding street food, I can provide some excellent recommendations. While a traditional "Gujarati Thali" might be rare to find outside of India, these options will ensure you have delicious and satisfying vegetarian meals.

Here are some highly-regarded establishments in Vienna that cater to vegetarian diets, focusing on quality and a pleasant dining experience:

### **For Authentic Indian Flavors (Closest to your Gujarati Thali Preference):**

1.  **Tulsi Indian Cuisine:**
    *   **Type:** Traditional Indian Restaurant
    *   **Why it's a fit:** Tulsi is highly regarded for its authentic Indian dishes with a strong emphasis on fresh ingredients. They have a comprehensive vegetarian menu, and the staff are generally very accommodating regarding specific dietary requests (e.g., no onion/garlic if preferred). The ambiance is refined and comfortable

## Agents with Memory

In [ ]:
from agno.agent import Agent
from agno.models.google import Gemini
from agno.tools.mem0 import Mem0Tools

In [ ]:
os.environ['MEM0_API_KEY'] = userdata.get('MEM0_API_KEY')

In [ ]:
agent = Agent(
    name="Personal Agent",
    model=Gemini(id="gemini-2.5-flash"),
    tools=[Mem0Tools(all=True)],
    user_id="tarun",
    description="Seasoned personal executive assistant that remembers and personalizes your interests.",
    instructions = """You have full access to memory operations. You can create, search, update, and delete memories.
        Proactively manage memories to provide the best user experience.""",
    markdown = True
)

In [ ]:
agent.print_response("I'm travelling to Europe- Vienna. Any food place recommendations?",
                     stream=True)

Output()

/usr/local/lib/python3.12/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

/usr/local/lib/python3.12/dist-packages/ipywidgets/widgets/widget_output.py:112: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  self.msg_id = ip.kernel._parent_header['header']['msg_id']